In [1]:
import os
import re
import shutil

def normalize_name(filename):
    """Remove cam15/cam18 and special characters to get base matching key."""
    name = os.path.splitext(filename)[0]
    name = re.sub(r'cam1[58]', '', name, flags=re.IGNORECASE)
    name = re.sub(r'[_\W]+', '', name.lower())  # remove non-alphanumeric
    return name

def find_and_copy_matches(stitched_dir, cam_dir, output_root):
    stitched_videos = [f for f in os.listdir(stitched_dir) if f.lower().endswith(('.mp4', '.avi', '.mov'))]
    cam_videos = [f for f in os.listdir(cam_dir) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

    # Normalize stitched names
    stitched_dict = {normalize_name(f): f for f in stitched_videos}

    os.makedirs(os.path.join(output_root, "cam15"), exist_ok=True)
    os.makedirs(os.path.join(output_root, "cam18"), exist_ok=True)

    for cam_vid in cam_videos:
        norm_cam = normalize_name(cam_vid)
        for norm_stitched in stitched_dict:
            if norm_stitched in norm_cam:
                src_path = os.path.join(cam_dir, cam_vid)
                if 'cam15' in cam_vid.lower():
                    dst_path = os.path.join(output_root, "cam15", cam_vid)
                elif 'cam18' in cam_vid.lower():
                    dst_path = os.path.join(output_root, "cam18", cam_vid)
                else:
                    continue  # Skip if neither cam15 nor cam18 is in the filename

                print(f"Copying: {cam_vid} --> {dst_path}")
                shutil.copy2(src_path, dst_path)
                break  # only match to one stitched video

# === Example usage ===
stitched_dir = r"D:\DeepSORT_ML2025\video_stitch\stitched_outputs"
cam_dir = r"D:\BehavSync_VK2023\DLC_labelling\Vesta-Vito\VV_pre_post_task_videos"
output_root = r"D:\DeepSORT_ML2025\video_stitch"
  # This will contain cam15/ and cam18/ subfolders

find_and_copy_matches(stitched_dir, cam_dir, output_root)


Copying: VV_ind2_cam15_pre.mp4 --> D:\DeepSORT_ML2025\video_stitch\cam15\VV_ind2_cam15_pre.mp4
Copying: VV_ind2_cam18_pre.mp4 --> D:\DeepSORT_ML2025\video_stitch\cam18\VV_ind2_cam18_pre.mp4
Copying: VV_pro2_cam15_pre.mp4 --> D:\DeepSORT_ML2025\video_stitch\cam15\VV_pro2_cam15_pre.mp4
Copying: VV_pro2_cam18_pre.mp4 --> D:\DeepSORT_ML2025\video_stitch\cam18\VV_pro2_cam18_pre.mp4
Copying: VV_rew4_cam15_post.mp4 --> D:\DeepSORT_ML2025\video_stitch\cam15\VV_rew4_cam15_post.mp4


KeyboardInterrupt: 

In [1]:
import os
import shutil

def collect_all_videos(input_root, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    used_names = set()

    for root, dirs, files in os.walk(input_root):
        for file in files:
            if file.endswith('.mp4'):
                original_path = os.path.join(root, file)

                # Generate a unique name using subfolder structure
                relative_path = os.path.relpath(original_path, input_root)
                flattened_name = relative_path.replace(os.sep, '__')

                # Ensure uniqueness in case two files have same flattened name
                base_name = os.path.splitext(flattened_name)[0]
                ext = os.path.splitext(flattened_name)[1]
                new_name = flattened_name
                counter = 1
                while new_name in used_names:
                    new_name = f"{base_name}_{counter}{ext}"
                    counter += 1
                used_names.add(new_name)

                # Copy the file
                destination_path = os.path.join(output_dir, new_name)
                shutil.copy2(original_path, destination_path)
                print(f"Copied: {original_path} -> {destination_path}")

    print(f"\n✅ All videos collected in: {output_dir}")

# === USAGE ===
input_root = r"D:\DeepSORT_ML2025\MaskedVideos_PerID_padded"
output_dir = r"D:\DeepSORT_ML2025\all_masked_vids"
collect_all_videos(input_root, output_dir)


Copied: D:\DeepSORT_ML2025\MaskedVideos_PerID_padded\cam10\JM_ind1pre\JM_ind1pre_ID1.mp4 -> D:\DeepSORT_ML2025\all_masked_vids\cam10__JM_ind1pre__JM_ind1pre_ID1.mp4
Copied: D:\DeepSORT_ML2025\MaskedVideos_PerID_padded\cam10\JM_ind1pre\JM_ind1pre_ID2.mp4 -> D:\DeepSORT_ML2025\all_masked_vids\cam10__JM_ind1pre__JM_ind1pre_ID2.mp4
Copied: D:\DeepSORT_ML2025\MaskedVideos_PerID_padded\cam10\JM_ind4pre\JM_ind4pre_ID1.mp4 -> D:\DeepSORT_ML2025\all_masked_vids\cam10__JM_ind4pre__JM_ind4pre_ID1.mp4
Copied: D:\DeepSORT_ML2025\MaskedVideos_PerID_padded\cam10\JM_ind4pre\JM_ind4pre_ID2.mp4 -> D:\DeepSORT_ML2025\all_masked_vids\cam10__JM_ind4pre__JM_ind4pre_ID2.mp4
Copied: D:\DeepSORT_ML2025\MaskedVideos_PerID_padded\cam10\JM_pro1post\JM_pro1post_ID1.mp4 -> D:\DeepSORT_ML2025\all_masked_vids\cam10__JM_pro1post__JM_pro1post_ID1.mp4
Copied: D:\DeepSORT_ML2025\MaskedVideos_PerID_padded\cam10\JM_pro1post\JM_pro1post_ID2.mp4 -> D:\DeepSORT_ML2025\all_masked_vids\cam10__JM_pro1post__JM_pro1post_ID2.mp4
Co

In [ ]:
import os
import cv2
import numpy as np
import csv

def draw_masked_boxes(video_path, csv_path, output_path_prefix, wiggle=7):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    bboxes_by_frame_id = {}
    with open(csv_path, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        if all(x in header for x in ['x1', 'y1', 'x2', 'y2']):
            idx_frame = header.index('frame')
            idx_id = header.index('id')
            idx_x1 = header.index('x1')
            idx_y1 = header.index('y1')
            idx_x2 = header.index('x2')
            idx_y2 = header.index('y2')
            for row in reader:
                if row[idx_x1] == '' or row[idx_y1] == '' or row[idx_x2] == '' or row[idx_y2] == '':
                    continue
                frame_i = int(row[idx_frame])
                id_i = int(row[idx_id])
                x1 = int(float(row[idx_x1])) - wiggle
                y1 = int(float(row[idx_y1])) - wiggle
                x2 = int(float(row[idx_x2])) + wiggle
                y2 = int(float(row[idx_y2])) + wiggle
                bboxes_by_frame_id.setdefault(frame_i, {}).setdefault(id_i, []).append([
                    max(0, x1), max(0, y1), min(width - 1, x2), min(height - 1, y2)
                ])
        else:
            idx_frame = header.index('frame')
            idx_id = header.index('id')
            idx_cx = header.index('cx')
            idx_cy = header.index('cy')
            bbox_w = 50 + 2 * wiggle
            bbox_h = 50 + 2 * wiggle
            for row in reader:
                if row[idx_cx] == '' or row[idx_cy] == '':
                    continue
                frame_i = int(row[idx_frame])
                id_i = int(row[idx_id])
                cx = float(row[idx_cx])
                cy = float(row[idx_cy])
                x1 = int(cx - bbox_w // 2)
                y1 = int(cy - bbox_h // 2)
                x2 = int(cx + bbox_w // 2)
                y2 = int(cy + bbox_h // 2)
                bboxes_by_frame_id.setdefault(frame_i, {}).setdefault(id_i, []).append([
                    max(0, x1), max(0, y1), min(width - 1, x2), min(height - 1, y2)
                ])

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out1 = cv2.VideoWriter(f'{output_path_prefix}_ID1.mp4', fourcc, fps, (width, height))
    out2 = cv2.VideoWriter(f'{output_path_prefix}_ID2.mp4', fourcc, fps, (width, height))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        white_bg = np.ones_like(frame, dtype=np.uint8) * 255
        for bbox in bboxes_by_frame_id.get(frame_idx, {}).get(1, []):
            x1, y1, x2, y2 = bbox
            white_bg[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
        out1.write(white_bg)

        white_bg = np.ones_like(frame, dtype=np.uint8) * 255
        for bbox in bboxes_by_frame_id.get(frame_idx, {}).get(2, []):
            x1, y1, x2, y2 = bbox
            white_bg[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
        out2.write(white_bg)

        frame_idx += 1

    cap.release()
    out1.release()
    out2.release()
    print(f"Finished masked videos for {os.path.basename(video_path)}")

def video_name_to_csv_folder_cam10_11(video_name):
    if '_cam10_' in video_name:
        base = video_name.replace('_cam10_', '_')
    elif '_cam11_' in video_name:
        base = video_name.replace('_cam11_', '_')
    else:
        base = video_name
    if base.endswith('_pre'):
        base = base[:-4] + 'pre'
    elif base.endswith('_post'):
        base = base[:-5] + 'post'
    return base

def video_name_to_csv_folder_cam15_18(video_name, cam_num):
    if cam_num == 15:
        base = video_name.replace('_cam15_', '_')
    elif cam_num == 18:
        base = video_name.replace('_cam18_', '_')
    else:
        base = video_name

    if base.endswith('_pre'):
        suffix = 'pre'
        base = base[:-4] + 'pre'
    elif base.endswith('_post'):
        suffix = 'post'
        base = base[:-5] + 'post'
    else:
        suffix = ''

    return f"{base}_from_{cam_num}_{suffix}"

def process_cam10_cam11(videos_root, csv_root, output_root):
    cam10_dir = os.path.join(videos_root, 'cam10')
    cam11_dir = os.path.join(videos_root, 'cam11')

    cam10_videos = [f for f in os.listdir(cam10_dir) if f.endswith('.mp4')]

    for video_filename in cam10_videos:
        base_name = os.path.splitext(video_filename)[0]
        csv_folder_name = video_name_to_csv_folder_cam10_11(base_name)
        csv_folder = os.path.join(csv_root, csv_folder_name)

        video_path_cam10 = os.path.join(cam10_dir, video_filename)
        video_filename_cam11 = video_filename.replace('_cam10_', '_cam11_')
        video_path_cam11 = os.path.join(cam11_dir, video_filename_cam11)

        left_csv = os.path.join(csv_folder, f"{csv_folder_name}_cam10_corrected.csv")
        right_csv = os.path.join(csv_folder, f"{csv_folder_name}_cam11_corrected.csv")

        out_folder_cam10 = os.path.join(output_root, 'cam10', csv_folder_name)
        out_folder_cam11 = os.path.join(output_root, 'cam11', csv_folder_name)
        os.makedirs(out_folder_cam10, exist_ok=True)
        os.makedirs(out_folder_cam11, exist_ok=True)

        if os.path.exists(video_path_cam10) and os.path.exists(left_csv):
            print(f"Processing cam10 video: {video_path_cam10}")
            draw_masked_boxes(video_path_cam10, left_csv, os.path.join(out_folder_cam10, csv_folder_name))
        else:
            print(f"Missing cam10 video or left CSV for {csv_folder_name}")
            print(f"  Expected video: {video_path_cam10}")
            print(f"  Expected CSV:   {left_csv}")

        if os.path.exists(video_path_cam11) and os.path.exists(right_csv):
            print(f"Processing cam11 video: {video_path_cam11}")
            draw_masked_boxes(video_path_cam11, right_csv, os.path.join(out_folder_cam11, csv_folder_name))
        else:
            print(f"Missing cam11 video or right CSV for {csv_folder_name}")
            print(f"  Expected video: {video_path_cam11}")
            print(f"  Expected CSV:   {right_csv}")

def find_csv_folder_for_cam15_18(csv_root, video_name_wo_ext, cam_num):
    target_folder = video_name_to_csv_folder_cam15_18(video_name_wo_ext, cam_num)
    full_path = os.path.join(csv_root, target_folder)
    return full_path if os.path.exists(full_path) else None

def process_cam15_or_18(cam_dir, csv_root, output_root, cam_num):
    for video_filename in os.listdir(cam_dir):
        if not video_filename.endswith('.mp4'):
            continue
        name_wo_ext = os.path.splitext(video_filename)[0]
        video_path = os.path.join(cam_dir, video_filename)

        csv_folder = find_csv_folder_for_cam15_18(csv_root, name_wo_ext, cam_num)
        if csv_folder is None:
            print(f"No CSV folder found for {name_wo_ext} in {csv_root} for cam{cam_num}")
            continue

        csv_path = os.path.join(csv_folder, f"{os.path.basename(csv_folder)}_tracking.csv")
        out_folder = os.path.join(output_root, f"cam{cam_num}", os.path.basename(csv_folder))
        os.makedirs(out_folder, exist_ok=True)

        if os.path.exists(video_path) and os.path.exists(csv_path):
            print(f"Processing cam{cam_num} video: {video_path}")
            draw_masked_boxes(video_path, csv_path, os.path.join(out_folder, os.path.basename(csv_folder)))
        else:
            print(f"Missing cam{cam_num} video or CSV for {os.path.basename(csv_folder)}")
            print(f"  Expected video: {video_path}")
            print(f"  Expected CSV:   {csv_path}")

def main():
    root_videos = r"D:\DeepSORT_ML2025\video_stitch"
    cam10_11_csv_root = r"D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked"
    cam15_csv_root = r"D:\cam15guide"
    cam18_csv_root = r"D:\cam18with_guide"
    output_root = r"D:\DeepSORT_ML2025\video_stitch\cam10_cam11_masked"

    print("Processing cam10 and cam11...")
    process_cam10_cam11(root_videos, cam10_11_csv_root, output_root)

    print("Processing cam15...")
    process_cam15_or_18(os.path.join(root_videos, 'cam15'), cam15_csv_root, output_root, cam_num=15)

    print("Processing cam18...")
    process_cam15_or_18(os.path.join(root_videos, 'cam18'), cam18_csv_root, output_root, cam_num=18)

if __name__ == "__main__":
    main()


Processing cam10 and cam11...
Missing cam10 video or left CSV for JM_ind1post
  Expected video: D:\DeepSORT_ML2025\video_stitch\cam10\JM_ind1_cam10_post.mp4
  Expected CSV:   D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1post\JM_ind1post_cam10_correct.csv
Processing cam11 video: D:\DeepSORT_ML2025\video_stitch\cam11\JM_ind1_cam11_post.mp4
Finished masked videos for JM_ind1_cam11_post.mp4
Missing cam10 video or left CSV for JM_ind1pre
  Expected video: D:\DeepSORT_ML2025\video_stitch\cam10\JM_ind1_cam10_pre.mp4
  Expected CSV:   D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1pre\JM_ind1pre_cam10_correct.csv
Processing cam11 video: D:\DeepSORT_ML2025\video_stitch\cam11\JM_ind1_cam11_pre.mp4


KeyboardInterrupt: 

In [11]:
import os
import cv2
import numpy as np
import csv

def draw_masked_boxes(video_path, csv_path, output_path_prefix, wiggle=7, shift_x=0, shift_y=0):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Cannot open video: {video_path}")
        return
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    bboxes_by_frame_id = {}
    with open(csv_path, 'r') as f:
        reader = csv.reader(f)
        header = next(reader)
        if all(x in header for x in ['x1', 'y1', 'x2', 'y2']):
            idx_frame = header.index('frame')
            idx_id = header.index('id')
            idx_x1 = header.index('x1')
            idx_y1 = header.index('y1')
            idx_x2 = header.index('x2')
            idx_y2 = header.index('y2')
            for row in reader:
                if row[idx_x1] == '' or row[idx_y1] == '' or row[idx_x2] == '' or row[idx_y2] == '':
                    continue
                frame_i = int(row[idx_frame])
                id_i = int(row[idx_id])
                x1 = int(float(row[idx_x1])) - wiggle
                y1 = int(float(row[idx_y1])) - wiggle
                x2 = int(float(row[idx_x2])) + wiggle
                y2 = int(float(row[idx_y2])) + wiggle
                bboxes_by_frame_id.setdefault(frame_i, {}).setdefault(id_i, []).append([
                    max(0, x1), max(0, y1), min(width - 1, x2), min(height - 1, y2)
                ])
        else:
            idx_frame = header.index('frame')
            idx_id = header.index('id')
            idx_cx = header.index('cx')
            idx_cy = header.index('cy')
            bbox_w = 50 + 2 * wiggle
            bbox_h = 50 + 2 * wiggle
            for row in reader:
                if row[idx_cx] == '' or row[idx_cy] == '':
                    continue
                frame_i = int(row[idx_frame])
                id_i = int(row[idx_id])
                cx = float(row[idx_cx])
                cy = float(row[idx_cy])
                x1 = int(cx - bbox_w // 2)
                y1 = int(cy - bbox_h // 2)
                x2 = int(cx + bbox_w // 2)
                y2 = int(cy + bbox_h // 2)
                bboxes_by_frame_id.setdefault(frame_i, {}).setdefault(id_i, []).append([
                    max(0, x1), max(0, y1), min(width - 1, x2), min(height - 1, y2)
                ])

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out1 = cv2.VideoWriter(f'{output_path_prefix}_ID1.mp4', fourcc, fps, (width, height))
    out2 = cv2.VideoWriter(f'{output_path_prefix}_ID2.mp4', fourcc, fps, (width, height))

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # ID 1 masked video (no shift)
        white_bg = np.ones_like(frame, dtype=np.uint8) * 255
        for bbox in bboxes_by_frame_id.get(frame_idx, {}).get(1, []):
            x1, y1, x2, y2 = bbox
            white_bg[y1:y2, x1:x2] = frame[y1:y2, x1:x2]
        out1.write(white_bg)

        # ID 2 masked video (apply shift)
        white_bg = np.ones_like(frame, dtype=np.uint8) * 255
        for bbox in bboxes_by_frame_id.get(frame_idx, {}).get(2, []):
            x1, y1, x2, y2 = bbox
            # Apply shift to all bounding boxes for cam11 videos (called with shift params)
            x1_s = max(0, x1 + shift_x)
            y1_s = max(0, y1 + shift_y)
            x2_s = min(width - 1, x2 + shift_x)
            y2_s = min(height - 1, y2 + shift_y)
            # Make sure shifted bbox dims match original bbox dims exactly to avoid broadcasting errors
            shifted_w = x2_s - x1_s
            shifted_h = y2_s - y1_s
            orig_w = x2 - x1
            orig_h = y2 - y1
            # Fix dims if off by 1 due to clamping
            if shifted_w != orig_w:
                if shifted_w < orig_w:
                    x2_s = x1_s + orig_w
                else:
                    x2_s = min(width - 1, x1_s + orig_w)
            if shifted_h != orig_h:
                if shifted_h < orig_h:
                    y2_s = y1_s + orig_h
                else:
                    y2_s = min(height - 1, y1_s + orig_h)
            white_bg[y1_s:y2_s, x1_s:x2_s] = frame[y1:y2, x1:x2]
        out2.write(white_bg)

        frame_idx += 1

    cap.release()
    out1.release()
    out2.release()
    print(f"Finished masked videos for {os.path.basename(video_path)}")

def video_name_to_csv_folder_cam10_11(video_name):
    if '_cam10_' in video_name:
        base = video_name.replace('_cam10_', '_')
    elif '_cam11_' in video_name:
        base = video_name.replace('_cam11_', '_')
    else:
        base = video_name
    if base.endswith('_pre'):
        base = base[:-4] + 'pre'
    elif base.endswith('_post'):
        base = base[:-5] + 'post'
    return base

def video_name_to_csv_folder_cam15_18(video_name, cam_num):
    if cam_num == 15:
        base = video_name.replace('_cam15_', '_')
    elif cam_num == 18:
        base = video_name.replace('_cam18_', '_')
    else:
        base = video_name

    if base.endswith('_pre'):
        suffix = 'pre'
        base = base[:-4] + 'pre'
    elif base.endswith('_post'):
        suffix = 'post'
        base = base[:-5] + 'post'
    else:
        suffix = ''

    return f"{base}_from_{cam_num}_{suffix}"

def process_cam10_cam11(videos_root, csv_root, output_root):
    cam10_dir = os.path.join(videos_root, 'cam10')
    cam11_dir = os.path.join(videos_root, 'cam11')

    cam10_videos = [f for f in os.listdir(cam10_dir) if f.endswith('.mp4')]

    for video_filename in cam10_videos:
        base_name = os.path.splitext(video_filename)[0]
        csv_folder_name = video_name_to_csv_folder_cam10_11(base_name)
        csv_folder = os.path.join(csv_root, csv_folder_name)

        video_path_cam10 = os.path.join(cam10_dir, video_filename)
        video_filename_cam11 = video_filename.replace('_cam10_', '_cam11_')
        video_path_cam11 = os.path.join(cam11_dir, video_filename_cam11)

        left_csv = os.path.join(csv_folder, f"{csv_folder_name}_cam10_correct.csv")
        right_csv = os.path.join(csv_folder, f"{csv_folder_name}_cam11_corrected.csv")

        out_folder_cam10 = os.path.join(output_root, 'cam10', csv_folder_name)
        out_folder_cam11 = os.path.join(output_root, 'cam11', csv_folder_name)
        os.makedirs(out_folder_cam10, exist_ok=True)
        os.makedirs(out_folder_cam11, exist_ok=True)

        if os.path.exists(video_path_cam10) and os.path.exists(left_csv):
            print(f"Processing cam10 video: {video_path_cam10}")
            draw_masked_boxes(video_path_cam10, left_csv, os.path.join(out_folder_cam10, csv_folder_name), shift_x=0, shift_y=0)
        else:
            print(f"Missing cam10 video or left CSV for {csv_folder_name}")
            print(f"  Expected video: {video_path_cam10}")
            print(f"  Expected CSV:   {left_csv}")

        if os.path.exists(video_path_cam11) and os.path.exists(right_csv):
            print(f"Processing cam11 video: {video_path_cam11}")
            # Shift ALL bounding boxes on cam11 by (-10, -10)
            draw_masked_boxes(video_path_cam11, right_csv, os.path.join(out_folder_cam11, csv_folder_name), shift_x=-30, shift_y=-10)
        else:
            print(f"Missing cam11 video or right CSV for {csv_folder_name}")
            print(f"  Expected video: {video_path_cam11}")
            print(f"  Expected CSV:   {right_csv}")

def find_csv_folder_for_cam15_18(csv_root, video_name_wo_ext, cam_num):
    target_folder = video_name_to_csv_folder_cam15_18(video_name_wo_ext, cam_num)
    full_path = os.path.join(csv_root, target_folder)
    return full_path if os.path.exists(full_path) else None

def process_cam15_or_18(cam_dir, csv_root, output_root, cam_num):
    for video_filename in os.listdir(cam_dir):
        if not video_filename.endswith('.mp4'):
            continue
        name_wo_ext = os.path.splitext(video_filename)[0]
        video_path = os.path.join(cam_dir, video_filename)

        csv_folder = find_csv_folder_for_cam15_18(csv_root, name_wo_ext, cam_num)
        if csv_folder is None:
            print(f"No CSV folder found for {name_wo_ext} in {csv_root} for cam{cam_num}")
            continue

        csv_path = os.path.join(csv_folder, f"{os.path.basename(csv_folder)}_tracking.csv")
        out_folder = os.path.join(output_root, f"cam{cam_num}", os.path.basename(csv_folder))
        os.makedirs(out_folder, exist_ok=True)

        if os.path.exists(video_path) and os.path.exists(csv_path):
            print(f"Processing cam{cam_num} video: {video_path}")
            draw_masked_boxes(video_path, csv_path, os.path.join(out_folder, os.path.basename(csv_folder)))
        else:
            print(f"Missing cam{cam_num} video or CSV for {os.path.basename(csv_folder)}")
            print(f"  Expected video: {video_path}")
            print(f"  Expected CSV:   {csv_path}")

def main():
    root_videos = r"D:\DeepSORT_ML2025\video_stitch"
    cam10_11_csv_root = r"D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked"
    cam15_csv_root = r"D:\cam15guide"
    cam18_csv_root = r"D:\cam18with_guide"
    output_root = r"D:\DeepSORT_ML2025\video_stitch\cam10_cam11_masked"

    print("Processing cam10 and cam11...")
    process_cam10_cam11(root_videos, cam10_11_csv_root, output_root)

    print("Processing cam15...")
    process_cam15_or_18(os.path.join(root_videos, 'cam15'), cam15_csv_root, output_root, cam_num=15)

    print("Processing cam18...")
    process_cam15_or_18(os.path.join(root_videos, 'cam18'), cam18_csv_root, output_root, cam_num=18)

if __name__ == "__main__":
    main()


Processing cam10 and cam11...
Missing cam10 video or left CSV for JM_ind1post
  Expected video: D:\DeepSORT_ML2025\video_stitch\cam10\JM_ind1_cam10_post.mp4
  Expected CSV:   D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1post\JM_ind1post_cam10_correct.csv
Processing cam11 video: D:\DeepSORT_ML2025\video_stitch\cam11\JM_ind1_cam11_post.mp4
Finished masked videos for JM_ind1_cam11_post.mp4
Missing cam10 video or left CSV for JM_ind1pre
  Expected video: D:\DeepSORT_ML2025\video_stitch\cam10\JM_ind1_cam10_pre.mp4
  Expected CSV:   D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1pre\JM_ind1pre_cam10_correct.csv
Processing cam11 video: D:\DeepSORT_ML2025\video_stitch\cam11\JM_ind1_cam11_pre.mp4


KeyboardInterrupt: 

### Sort CSVs

In [13]:
import os
import shutil

# Set your root directories for each camera here
root_dirs = {
    "cam10": r"D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked",
    "cam11": r"D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked",
    "cam15": r"D:\DeepSORT_ML2025\video_stitch\cam15_tracked",
    "cam18": r"D:\DeepSORT_ML2025\video_stitch\cam15_tracked",
}

# Destination base folder to hold the 6 main folders
dest_base = r"D:\DeepSORT_ML2025\video_stitch\sorted_csv"


# The 6 main folders and camera subfolders to create
main_folders = ["ind_pre", "ind_post", "pro_pre", "pro_post", "rew_pre", "rew_post"]
cam_subfolders = ["cam10", "cam11", "cam15", "cam18"]
csv_subfolders = ["raw", "corrected", "interpolated"]

# Create full folder structure
for main_f in main_folders:
    for cam_f in cam_subfolders:
        for csv_f in csv_subfolders:
            os.makedirs(os.path.join(dest_base, main_f, cam_f, csv_f), exist_ok=True)

# Helper to determine main folder from filename
def get_main_folder(filename):
    filename = filename.lower()
    category = None
    timing = None
    
    for c in ["ind", "pro", "rew"]:
        if c in filename:
            category = c
            break
    for t in ["pre", "post"]:
        if t in filename:
            timing = t
            break
    
    if category and timing:
        return f"{category}_{timing}"
    return None

# Helper to determine CSV subfolder from filename
def get_csv_subfolder(filename):
    filename = filename.lower()
    if "corrected" in filename:
        return "corrected"
    elif "interpol" in filename:
        return "interpolated"
    else:
        return "raw"

# Go through each camera root directory
for cam, root_dir in root_dirs.items():
    for dirpath, _, files in os.walk(root_dir):
        for f in files:
            if not f.lower().endswith(".csv"):
                continue
            if cam not in f.lower():
                continue
            
            main_folder = get_main_folder(f)
            if main_folder:
                csv_folder = get_csv_subfolder(f)
                src_path = os.path.join(dirpath, f)
                dest_path = os.path.join(dest_base, main_folder, cam, csv_folder, f)
                shutil.copy2(src_path, dest_path)
                print(f"Copied {src_path} -> {dest_path}")


Copied D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1post\JM_ind1post_cam10.csv -> D:\DeepSORT_ML2025\video_stitch\sorted_csv\ind_post\cam10\raw\JM_ind1post_cam10.csv
Copied D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1post\JM_ind1post_cam10_corrected.csv -> D:\DeepSORT_ML2025\video_stitch\sorted_csv\ind_post\cam10\corrected\JM_ind1post_cam10_corrected.csv
Copied D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1post\JM_ind1post_centre_interpol_cam10.csv -> D:\DeepSORT_ML2025\video_stitch\sorted_csv\ind_post\cam10\interpolated\JM_ind1post_centre_interpol_cam10.csv
Copied D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1pre\JM_ind1pre_cam10.csv -> D:\DeepSORT_ML2025\video_stitch\sorted_csv\ind_pre\cam10\raw\JM_ind1pre_cam10.csv
Copied D:\DeepSORT_ML2025\video_stitch\cam10_cam11_tracked\JM_ind1pre\JM_ind1pre_cam10_corrected.csv -> D:\DeepSORT_ML2025\video_stitch\sorted_csv\ind_pre\cam10\corrected\JM_ind1pre_cam10_corrected.csv
Copied D:\DeepS